In [ ]:
import os
import sys
import time as t_util
import numpy as np
import pandas as pd
import yaml
import cftime
import xarray as xr


## Define folders

In [ ]:
#Read main paths
with open('../path_main.txt', 'r') as file:   path_main = file.read()
with open('../path_CMIP5.txt', 'r') as file:  path_cmip5 = file.read()
    
dir_CMIP5   = path_cmip5
dir_scripts = f'{path_main}Scripts/'
dir_names   = f'{path_main}Scripts/Model_lists/'
dir_JJA     = f'{path_main}Data/CMIP5/JJA/'
dir_Yearmax = f'{path_main}Data/CMIP5/Yearmax/'
if not os.path.exists(dir_JJA):      os.mkdir(dir_JJA)
if not os.path.exists(dir_Yearmax):  os.mkdir(dir_Yearmax)


## Prepare variables and parameters

In [ ]:
#Define models and RCPs which should be used
all_models = dict()
all_models['rcp85'] = []
with open(dir_names + 'Models_CMIP5_RCP85.txt', 'r') as filehandle:
    for line in filehandle:
        all_models['rcp85'].append(line[:-1])

#Define time periods
time_periods = [[1981, 2010], [2070, 2099]]

scenarios = ['historical', 'rcp85']

variables = ['tasmax']#, 'tasmin']
vars_out = dict()
vars_out['tasmax'] = 'TXx'
vars_out['tasmin'] = 'TNx'


## Prepare CMIP5 data

In [ ]:
#Loop over models
for model in all_models['rcp85']:
    
    print(model)

    #Define members
    if model=='EC-EARTH':      members = ['r1i1p1', 'r12i1p1']
    elif model=='MPI-ESM-LR':  members = ['r1i1p1', 'r2i1p1', 'r3i1p1']
    else:                      members = ['r1i1p1']        

    #Loop over members
    for member in members:  

        #Loop over variables
        for variab in variables:

            print(variab, end=', ')

            #Loop over scenarios
            fnames_merge = []
            for scen in scenarios:

                #Get file names
                dir_data = dir_CMIP5 + scen + '/'
                fnames = [dir_data + file for file in os.listdir(dir_data) if model + '_' in file and member in file and variab in file and scen in file and '_day_' in file]
                fnames = sorted(fnames)
                if len(fnames)==0: sys.exit('Filename not uniquely defined')

                #Add to list for all files for merging
                fnames_merge = fnames_merge + fnames

            #Merge single files to one large file
            file_merge = dir_Yearmax + 'CMIP5_merged_' + variab + '_' + scen + "_" + (model) + '_Yearmax_tmp.nc'
            if os.path.exists(file_merge): os.remove(file_merge)
            os.system('cdo mergetime ' + " ".join(fnames_merge) + " " + file_merge)

            #Open data set and rename variable
            data = xr.open_dataset(file_merge, use_cftime=True)

            #Loop over time periods
            for time_per in time_periods:

                #Create slice for selecting data
                time_sel = slice(str(time_per[0]), str(time_per[1]))

                #Select time
                data_sel = data.sel(time=time_sel)

                #Check units of huss
                check1 = data_sel[variab].isel(time=slice(0, 100)).mean()
                check2 = data_sel[variab].isel(time=slice(-100, -1)).mean()
                if (variab=='huss') and ((check1>0.01) or (check1<0.0001) or (check2>0.01) or (check2<0.0001)):
                    sys.exit('Check units of ' + vars_out[variab])

                #Select summer
                sel_JJA  = (data_sel.time.dt.month>=6) & (data_sel.time.dt.month<=8)
                data_JJA = data_sel.isel(time=sel_JJA)

                #Calculate summer mean and yearly maximum
                data_JJA_mean = data_JJA.mean('time')
                data_Yearmax  = data_sel.resample(time='1Y').max()
                data_Yearmax  = data_Yearmax.rename({variab: vars_out[variab]})

                #Creat output directory
                dir_JJA_out  =  dir_JJA + variab + '/'
                dir_Ymax_out =  dir_Yearmax + vars_out[variab] + '/'
                if not os.path.exists(dir_JJA_out):   os.mkdir(dir_JJA_out)
                if not os.path.exists(dir_Ymax_out):  os.mkdir(dir_Ymax_out)      

                #Create output file names
                output_str = "_" + model + '_' + member + '_' + "-".join(scenarios) +  "_" + time_sel.start + "-" + time_sel.stop + ".nc"
                fname_JJA_mean = dir_JJA_out + variab + "_JJA-mean" + output_str
                fname_Yearmax  = dir_Ymax_out + vars_out[variab] + "_Yearmax" + output_str

                #Save in file
                if os.path.exists(fname_JJA_mean): os.remove(fname_JJA_mean)
                if os.path.exists(fname_Yearmax): os.remove(fname_Yearmax)
                data_JJA_mean.to_netcdf(fname_JJA_mean)
                data_Yearmax.to_netcdf(fname_Yearmax)

            #Remove temporarily merged file
            os.remove(file_merge)
